list google bucket url map?


In [ ]:
var path = require('path');
var importer = require('../Core');
var authorizeGoogle = importer.import('authorize google service');

function listUrlMaps(project, urlMap) {
    var params = {};
    if(urlMap) {
        params.filter = `name="${urlMap}"`
    }
    return authorizeGoogle()
        .then(client => client.request({
            url: `https://www.googleapis.com/compute/v1/projects/${project}/global/urlMaps`,
            params
        }))
        .then(res => {
            return (res.data.items || []).reduce((obj, cur) => {
                obj[cur.name] = {
                    hostRules: cur.hostRules,
                    pathMatchers: cur.pathMatchers,
                    fingerprint: cur.fingerprint
//    defaultService: 'https://www.googleapis.com/compute/v1/projects/spahaha-ea443/global/backendServices/web-map-backend-service',
                };
                return obj;
            }, {})
        });
}

function listTargetProxies(project, urlMap) {
    var params = {};
    if(urlMap) {
        params.filter = `urlMap="https://www.googleapis.com/compute/v1/projects/${project}/global/urlMaps/${urlMap}"`
    }
    return authorizeGoogle()
        .then(client => client.request({
            url: `https://www.googleapis.com/compute/v1/projects/${project}/global/targetHttpsProxies`,
            params
        }))
        .then(res => (res.data.items || []).reduce((obj, p) => {
            obj[p.name] = p.sslCertificates;
            return obj
        }, {}));
}

function listGlobalForwards(project, proxy, ip) {
    var params = {};
    if(proxy) {
        params['filter'] = `target="https://www.googleapis.com/compute/v1/projects/${project}/global/targetHttpsProxies/${proxy}"`
    }
    if(ip) {
        params['filter'] = `IPAddress="https://www.googleapis.com/compute/v1/projects/${project}/global/addresses/${ip}"`;
    }
    return authorizeGoogle()
        .then(client => client.request({
            url: `https://www.googleapis.com/compute/v1/projects/${project}/global/forwardingRules`,
            params
        }))
        .then(res => {
            return (res.data.items || []).reduce((obj, cur) => {
                obj[cur.name] = {
                    IPAddress: cur.IPAddress,
                    target: cur.target
                }
                return obj;
            }, {})
        });
}

function listBackendBuckets(project, bucketName) {
    var params = {};
    if(bucketName) {
        params.filter = `bucketName="${bucketName}"`;
    }
    return authorizeGoogle()
        .then(client => client.request({
            url: `https://www.googleapis.com/compute/v1/projects/${project}/global/backendBuckets`,
            params
        }))
        .then(res => {
            return (res.data.items || []).reduce((obj, cur) => {
                obj[cur.name] = cur.bucketName;
                return obj;
            }, {})
        });
}

function listSslCertificates(project, certName) {
    var params = {};
    if(certName) {
        params.filter = `name="${certName}"`;
    }
    return authorizeGoogle()
        .then(client => client.request({
            url: `https://www.googleapis.com/compute/v1/projects/${project}/global/sslCertificates`,
            params
        }))
        .then(res => {
            return (res.data.items || []).reduce((obj, cur) => {
                obj[cur.name] = cur.description;
                return obj;
            }, {})
        });
}

module.exports = {
    listUrlMaps,
    listTargetProxies,
    listGlobalForwards,
    listBackendBuckets,
    listSslCertificates
};


test list url map?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var {
    listUrlMaps,
    listTargetProxies,
    listGlobalForwards,
    listBackendBuckets
} = importer.import('list google bucket url map');
var project = 'spahaha-ea443';
var map = 'web-map';
var proxy = 'web-map-target-proxy-5';

describe('listing subdomains from load balancer', () => {
    
    it('should list buckets maps', () => {
        return listUrlMaps(project)
            .then(items => {
                console.log(items[Object.keys(items)[0]].hostRules);
                console.log(items[Object.keys(items)[0]].pathMatchers);
                console.log(items[Object.keys(items)[0]].pathMatchers[0].pathRules);
                assert(Object.keys(items).length > 0, 'should have a url mapper');
                
                return listTargetProxies(
                    project,
                    map);
            })
            .then(proxies => {
                console.log(proxies);
                assert(proxies.length > 0, 'should have a proxy');
            
                return listGlobalForwards(
                    project,
                    proxy);
            })
            .then(rules => {
                console.log(rules);
                assert(Object.keys(rules).length > 0, 'should have a rule');
            
                return listBackendBuckets(project);
            })
            .then(buckets => {
                console.log(buckets);
                assert(Object.keys(buckets).length > 0, 'should have a bucket');
            });
    })
    
    
})


how to wait for cloud operation?

wait google cloud operation?


In [ ]:
var importer = require('../Core');
var authorizeGoogle = importer.import('authorize google service');

function waitForOperation(operation) {
    return authorizeGoogle()
        .then(client => client.request({
            url: `${operation}`,
        }))
        .then(res => {
            if(res.data.status != 'RUNNING') {
                return Promise.resolve(res.data.targetLink);
            }
            console.log(`waiting for ${operation}`);
            return new Promise(resolve => setTimeout(resolve, 500))
                .then(() => waitForOperation(operation));
        })
  
}

module.exports = waitForOperation;

add a google bucket web map?


In [ ]:
var importer = require('../Core');
var authorizeGoogle = importer.import('authorize google service');
var waitForOperation = importer.import('wait google cloud operation');
var uuid = require('uuid/v1');
var {
    listBackendBuckets,
    listTargetProxies,
    listGlobalForwards,
    listUrlMaps,
    listSslCertificates
} = importer.import('list google bucket url map');
var createBucket = importer.import('create storage bucket');


function safeName(name) {
    return name.replace(/[^a-z0-9\-]/ig, '-').substr(0, 50).toLowerCase();
}

function insertBackendBucket(project, bucketName) {
    // TODO: check for creation of bucket here?
    var name = 'bb-' + uuid().substr(0, 5) + '-' + safeName(bucketName);
    return createBucket(project, bucketName)
        .then(() => listBackendBuckets(project, bucketName))
        .then(buckets => {
            if(Object.keys(buckets).length > 0) {
                name = Object.keys(buckets)[0];
                console.log(`Bucket ${bucketName} already mapped`);
                return Promise.resolve(Object.keys(buckets)[0]);
            }
            console.log(`Creating a backend for ${bucketName}`);
            return authorizeGoogle()
                .then(client => client.request({
                    method: 'POST',
                    url: `https://www.googleapis.com/compute/v1/projects/${project}/global/backendBuckets`,
                    data: {
                        name: name,
                        bucketName: bucketName,
                        enableCdn: false,
                        kind: 'compute#backendBucket'
                    }
                }))
                .then(backend => {
                    return waitForOperation(backend.data.selfLink);
                })
        })
        .then(() => name);
}

function addSslCertificate(project, bucketName) {
    return authorizeGoogle()
        .then(client => client.request({
            method: 'POST',
            url: `https://www.googleapis.com/compute/beta/projects/${project}/global/sslCertificates`,
            data: {
                name: 'cert-' + safeName(bucketName),
                description: bucketName,
                managed: {domains: [bucketName]},
                type: 'MANAGED'
//                        sslPolicy: 'https://www.googleapis.com/compute/v1/projects/spahaha-ea443/global/sslPolicies/my-ssl-policy'
            }
        }))
        .then(ssl => {
            return waitForOperation(ssl.data.selfLink);
        })
        .then(() => 'cert-' + safeName(bucketName))
}

function insertSslCertificate(project, bucketName) {
    return listSslCertificates(project, 'cert-' + safeName(bucketName))
        .then(ssl => {
            if(Object.keys(ssl).length > 0) {
                console.log(`SSL ${bucketName} already exists`);
                return Promise.resolve(Object.keys(ssl)[0]);
            }
            console.log(`Creating an SSL cert for ${bucketName}`);
            return addSslCertificate(project, bucketName);
        })
}

function addTargetHttpsProxy(project, urlMap, bucketName) {
    // TODO: get SSL cert first because it's require by proxy
    var sslCertificate;
    var proxyName = 'thp-' + uuid().substr(0, 5) + '-' + safeName(bucketName);
    
    return insertSslCertificate(project, bucketName)
        .then(ssl => {
            sslCertificate = ssl;
        })
        .then(() => authorizeGoogle())
        .then(client => client.request({
            method: 'POST',
            url: `https://www.googleapis.com/compute/v1/projects/${project}/global/targetHttpsProxies`,
            data: {
                name: proxyName,
                urlMap: `https://www.googleapis.com/compute/v1/projects/${project}/global/urlMaps/${urlMap}`,
                sslCertificates: [
                    `https://www.googleapis.com/compute/v1/projects/${project}/global/sslCertificates/${sslCertificate}`
                ],
//              sslPolicy: 'https://www.googleapis.com/compute/v1/projects/spahaha-ea443/global/sslPolicies/my-ssl-policy'
            }
        }))
        .then(proxy => {
            return waitForOperation(proxy.data.selfLink);
        })
        .then(() => proxyName)
}

function insertTargetHttpsProxy(project, urlMap, bucketName) {
    // check for a proxy with an sslCert matching our project name
    return listTargetProxies(project)
        .then(proxies => {
            var matches = Object.keys(proxies)
                .filter(k => proxies[k]
                        .filter(cert => cert.includes(`https://www.googleapis.com/compute/v1/projects/${project}/global/sslCertificates/cert-${safeName(bucketName)}`)).length > 0);
            // TODO: is using sheet-to-web host, just add the certificate to existing proxy
            // support for SNI? https://www.ietf.org/rfc/rfc4366.txt
            
            if(matches.length > 0) {
                console.log(`Proxy ${bucketName} already exists`);
                return Promise.resolve(Object.keys(proxies)[0]);
            }
            console.log(`Creating a proxy for ${bucketName}`);
            return addTargetHttpsProxy(project, urlMap, bucketName);
        })
}

function addGlobalForward(project, ip, urlMap, bucketName, targetHttpsProxy) {
    var name = 'gfr-' + uuid().substr(0, 5) + '-' + safeName(urlMap)

    return authorizeGoogle()
        .then(client => client.request({
            method: 'POST',
            url: `https://www.googleapis.com/compute/v1/projects/${project}/global/forwardingRules`,
            data: {
                name: name,
                IPAddress: `https://www.googleapis.com/compute/v1/projects/${project}/global/addresses/${ip}`,
                IPProtocol: 'TCP',
                portRange: '443-443',
                target: `https://www.googleapis.com/compute/v1/projects/${project}/global/targetHttpsProxies/${targetHttpsProxy}`,
                loadBalancingScheme: 'EXTERNAL',
                networkTier: 'PREMIUM'
            }
        }))
        .then(rule => {
            return waitForOperation(rule.data.selfLink);
        })
        .then(() => name);
}

function insertGlobalForward(project, ip, urlMap, bucketName) {

    // get proxy first because it's require by forwardRule
    var targetHttpsProxy;
    return insertTargetHttpsProxy(project, urlMap, bucketName)
        .then(proxy => (targetHttpsProxy = proxy))
        .then(() => listGlobalForwards(project, targetHttpsProxy))
        .then(forwards => {
            if(Object.keys(forwards).length > 0) {
                console.log(`Forward for ${bucketName} already exists`);
                return Promise.resolve(Object.keys(forwards)[0]);
            }
            console.log(`Creating a forward for ${bucketName}`);
            return addGlobalForward(project, ip, urlMap, bucketName, targetHttpsProxy);
        })
}

function updateUrlMap(project, urlMap, bucketName) {
    return listUrlMaps(project, urlMap)
        // add a global forward if the ip isn't already set up
        .then(maps => {
            if(typeof maps[urlMap] == 'undefined') {
                // TODO: create the map
                throw new Error('Implement url map create');
            }
            if(maps[urlMap].hostRules.filter(hr => hr.hosts.filter(h => h == bucketName).length > 0).length > 0) {
                console.log(`Map host rule ${bucketName} already exists`);
                return Promise.resolve(urlMap);
            }
            console.log(`Map ${urlMap} already exists, adding host rule`);
            var defaultService;
            var pathMatcherName = 'pm-' + uuid().substr(0, 5) + '-' + safeName(bucketName);
            return insertBackendBucket(project, bucketName)
                .then(bn => (defaultService = bn))
                .then(() => authorizeGoogle())
                .then(client => client.request({
                    method: 'PATCH',
                    url: `https://www.googleapis.com/compute/v1/projects/${project}/global/urlMaps/${urlMap}`,
                    data: {
                        fingerprint: maps[urlMap].fingerprint,
                        hostRules: maps[urlMap].hostRules.concat([{
                            hosts: [bucketName],
                            pathMatcher: pathMatcherName
                        }]),
                        pathMatchers: maps[urlMap].pathMatchers.concat([{
                            name: pathMatcherName,
                            defaultService: `https://www.googleapis.com/compute/v1/projects/${project}/global/backendBuckets/${defaultService}`,
                            pathRules: ['/', '/*']
                        }]),
                    }
                }))
                .then(path => {
                    return waitForOperation(path.data.selfLink);
                })
                .then(map => pathMatcherName)
        });
}

module.exports = {
    insertBackendBucket,
    insertGlobalForward,
    updateUrlMap
};


test global forward?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var {listGlobalForwards} = importer.import('list global bucket url map');
var project = 'spahaha-ea443';
var proxy = 'https://www.googleapis.com/compute/v1/projects/spahaha-ea443/global/targetHttpsProxies/thp-0c28d-marketing-site-7b81f-sheet-to-web-com';

describe('listing global forwards', () => {
    it('should list forward', () => {
        return listGlobalForwards(project, proxy)
            .then(ip => {
                console.log(ip);
                assert(Object.keys(ip).length > 0, 'should list forward');
            })
    }).timeout(60000)
    
});


test bucket web map?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var {
    insertBackendBucket,
    insertGlobalForward,
    updateUrlMap
} = importer.import('add google bucket web map');
var addIP = importer.import('check dns');
var project = 'spahaha-ea443';
var domain = 'sheet-to-web.sheet-to-web.com';
var urlMap = 'web-map';

describe('adding a subdomain to a load balancer', () => {
    it('should add a bucket backend', () => {
        return insertBackendBucket(project, domain)
            .then(bucketName => {
                assert(bucketName.length > 0, 'should have added a backend');
            })
    }).timeout(60000)
    
    it('should add a frontend to load balancer', () => {
        // TODO: check for A record in DNS to get ip
        return addIP(project, domain)
            .then(ip => insertGlobalForward(project, ip, urlMap, domain))
            .then(bucketName => {
                assert(bucketName.length > 0, 'should have added a forward');
            })
    }).timeout(60000)
    
    it('should update url map on load balancer', () => {
        return updateUrlMap(project, urlMap, domain)
            .then(map => {
                assert(map.length > 0, 'should have updated the map');
            })
    }).timeout(60000)
})


how to check dns?


In [ ]:
var dns = require('dns');
var util = require('util');
var uuid = require('uuid/v1');
var importer = require('../Core');
var authorizeGoogle = importer.import('authorize google service');
var {listGlobalForwards} = importer.import('list google bucket url map');
var waitForOperation = importer.import('wait cloud operation');

var host = 'sheet-to-web.com';

function safeName(name) {
    return name.replace(/[^a-z0-9\-]/ig, '-').substr(0, 50).toLowerCase();
}

function createZone(client, project, subdomain) {
    var zone;
    return client.request({
            url: `https://www.googleapis.com/dns/v1/projects/${project}/managedZones`,
            params: {
                dnsName: subdomain.includes(host) ? (host + '.') : (subdomain + '.')
            }
        })
        .then(res => {
            if(res.data.managedZones.length > 0) {
                zone = res.data.managedZones[0].name;
                console.log(`Zone ${subdomain} already mapped`);
                return Promise.resolve(res.data.managedZones[0].name);
            }
            zone = 'mz-' + uuid().substr(0, 5) + '-' + safeName(subdomain);
            console.log(`Creating a zone for ${subdomain}`)
            return client.request({
                method: 'POST',
                url: `https://www.googleapis.com/dns/v1/projects/${project}/managedZones`,
                data: {
                    name: zone,
                    dnsName: subdomain + '.',
                    description: '',
                    visibility: 'public',
                    dnssecConfig: {
                        state: 'on',
                        kind: 'dns#managedZoneDnsSecConfig'
                    }
                }
            })
        })
        .then(() => zone)
}

function insertAddress(client, project, addrs, subdomain) {
    var zone;
    var name = 'ip-' + uuid().substr(0, 5) + '-' + safeName(subdomain);
    var address;
    var records;
    var defaultRecord;
    
    return createZone(client, project, subdomain)
        // TODO: check for unused ips
        .then(z => (zone = z))
        .then(() => {
            return client.request({
                url: `https://www.googleapis.com/dns/v1/projects/${project}/managedZones/${zone}/rrsets`
            })
        })
        .then(res => {
            defaultRecord = res.data.rrsets.filter(r => r.type == 'A' && r.name == host + '.')[0];
            records = res.data.rrsets.filter(r => r.type == 'A' && r.name == subdomain + '.');
        })
        // check if there is an ip unassigned
        .then(() => listGlobalForwards(project))
        .then(forwards => {
            // match existing dns records
            var match = Object.keys(addrs)
                // use sheet-to-web host ip address
                .filter(a => subdomain.includes(host)
                        ? defaultRecord.rrdatas[0] == addrs[a]
                        : records.filter(r => r.rrdatas[0] == addrs[a]).length > 0)
            if(match.length > 0) {
                name = match[0];
                address = addrs[match[0]];
                console.log(`Matched IP ${match[0]} already exists`);
                return Promise.resolve(match[0]);
            }
        
            var unused = Object.keys(addrs)
                .filter(a => Object.values(forwards)
                        .map(gf => gf.IPAddress)
                        .filter(ip => ip == addrs[a]).length == 0);
            if(unused.length > 0) {
                name = unused[0];
                address = addrs[unused[0]];
                console.log(`New unused IP ${unused[0]} already exists`);
                return Promise.resolve(unused[0]);
            }
        
            console.log(`Creating address for ${subdomain}`)
            return client.request({
                method: 'POST',
                url: `https://www.googleapis.com/compute/v1/projects/${project}/global/addresses`,
                data: {
                    name: name,
//                    ipVersion: 'IPV6',
                    networkTier: 'PREMIUM'
                }
            })
                .then(res => {
                    return waitForOperation(res.data.selfLink);
                })
                .then(target => {
                    return client.request({
                        url: target
                    })
                })
                .then(res => {
                    address = res.data.address;
                })
        })
        // update zone with A record
        .then(() => {
            return client.request({
                method: 'POST',
                url: `https://www.googleapis.com/dns/v1/projects/${project}/managedZones/${zone}/changes`,
                data: {
                    additions: [
                        {
                            kind: 'dns#resourceRecordSet',
                            name: subdomain + '.',
                            type: 'A',
                            ttl: 3600,
                            rrdatas: [address]
                        }
                    ],
                    deletions: records
                }
            })
        })
        .then(() => name)
    /*
    {
  "kind": "dns#resourceRecordSet",
  "name": "example.com.",
  "rrdatas": [
      "1.2.3.4"
  ],
  "ttl": 86400,
  "type": "A"
}
AAAA	
IPv6 Address record, which is used to map a host name to an IPv6 address.

Example of the resource record set representation:

{
  "kind": "dns#resourceRecordSet",
  "name": "example.com.",
  "rrdatas": [
      "2607:f8b0:400a:801::1005"
  ],
  "ttl": 86400,
  "type": "AAAA"
}
*/
}

function addIP(project, subdomain) {
    var client, addresses;
    return util.promisify(dns.lookup)(subdomain)
        .then((addrs, family) => {
            addresses = [addrs.address];
        })
        .catch(up => {
            if(up.message.includes('ENOTFOUND')) {
                addresses = [];
                console.log(up.message);
                return;
            }
            throw up
        })
        .then(() => authorizeGoogle())
        .then(c => (client = c).request({
            url: `https://www.googleapis.com/compute/v1/projects/${project}/global/addresses`,
        }))
        .then(res => {
            return (res.data.items || []).reduce((obj, addr) => {
                obj[addr.name] = addr.address;
                return obj
            }, {});
        })
        .then(addrs => {
            var matching = Object.keys(addrs).filter(a => addresses.indexOf(addrs[a]) > -1);
            if(matching.length > 0) {
                console.log(`domain ${subdomain} already exists, ip address ${matching[0]}`)
                return Promise.resolve(matching[0]);
            }
            return insertAddress(client, project, addrs, subdomain);
        });
}

module.exports = addIP;


test check dns?

In [ ]:
var assert = require('assert');
var importer = require('../Core');
var addIP = importer.import('check dns');
var project = 'spahaha-ea443';
var bucketName = 'sheet-to-web.sheet-to-web.com';

describe('adding an external ip', () => {
    it('should add an ip', () => {
        return addIP(project, bucketName)
            .then(ip => {
                console.log(ip);
                assert(ip.length > 0, 'should have added an ip');
            })
    }).timeout(60000)
    
});


invalidate cdn cache?


how to authorize google services?

authorize google service?


In [ ]:
var fs = require('fs');
var path = require('path');
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var credentials;
if(fs.existsSync('./spahaha-ea443-a78ab2269985.json')) {
    credentials = path.resolve('./spahaha-ea443-a78ab2269985.json');
} else {
    credentials = path.join(PROFILE_PATH, '.credentials/spahaha-ea443-a78ab2269985.json');
}

var {GoogleAuth} = require('google-auth-library');
var GOOGLE_AUTH_SCOPE = [
    'https://www.googleapis.com/auth/compute',
    'https://www.googleapis.com/auth/cloud-platform'
];

function authorizeGoogle() {
    return new GoogleAuth({
        keyFile: credentials,
        scopes: GOOGLE_AUTH_SCOPE
    }).getClient(/* options here are always ignored b/c cache */)
}

module.exports = authorizeGoogle;


### setup backend for domain



#### the code

sheet backend handler?

setup sheet backend?


In [ ]:
var importer = require('../Core');
var getSheet = importer.import('get sheet purchases');
//var purchaseId = '1kWjkjLGxQyzFUzRLBk3LpcjPW3UjcaF-PBMDX_3hZfM';
var addIP = importer.import('check dns');
var {
    insertBackendBucket,
    insertGlobalForward,
    updateUrlMap
} = importer.import('add google bucket web map');

var urlMap = 'web-map';

function setupBackend(link, domain) {
    var project;

    return getSheet(link, domain)
        .then(match => {
            project = match.project;
            domain = domain || match.bucket;
            return addIP(project, domain);
        })
        .then(ip => insertGlobalForward(project, ip, urlMap, domain))
        .then(() => insertBackendBucket(project, domain))
        .then(() => updateUrlMap(project, urlMap, domain))
        .then(() => domain)
}

module.exports = setupBackend;

